In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/hotel-booking/hotel_booking.csv
/kaggle/input/covid-19/.gitignore
/kaggle/input/covid-19/README.md
/kaggle/input/covid-19/datapackage.json
/kaggle/input/covid-19/scripts/process_us.py
/kaggle/input/covid-19/scripts/process_worldwide.py
/kaggle/input/covid-19/scripts/requirements.txt
/kaggle/input/covid-19/scripts/update_datapackage.py
/kaggle/input/covid-19/.github/workflows/actions.yml
/kaggle/input/covid-19/data/reference.csv
/kaggle/input/covid-19/data/countries-aggregated.csv
/kaggle/input/covid-19/data/us_simplified.csv
/kaggle/input/covid-19/data/key-countries-pivoted.csv
/kaggle/input/covid-19/data/us_deaths.csv
/kaggle/input/covid-19/data/time-series-19-covid-combined.csv
/kaggle/input/covid-19/data/worldwide-aggregate.csv
/kaggle/input/covid-19/data/us_confirmed.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('../input/hotel-booking/hotel_booking.csv')
df.sample(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
32060,Resort Hotel,0,3,2017,January,2,14,0,1,1,...,Transient-Party,39.0,0,0,Check-Out,2017-01-15,Diana Hardy,Diana_Hardy@yandex.com,138-881-7352,************2093
79927,City Hotel,0,21,2015,November,45,5,0,2,2,...,Transient,76.0,0,0,Check-Out,2015-11-07,Michelle Russell,Michelle.Russell@yandex.com,909-811-4239,************2504
65742,City Hotel,1,92,2017,April,14,7,1,2,2,...,Transient,99.0,0,0,Canceled,2017-01-05,Samuel Jones,Samuel_Jones@aol.com,370-521-4133,************1651
19223,Resort Hotel,1,92,2016,February,9,26,1,2,2,...,Transient,79.0,0,0,Canceled,2015-12-10,Amber Gomez,AmberGomez@aol.com,843-418-3308,************6018
106271,City Hotel,0,7,2017,February,8,23,0,1,1,...,Transient,98.0,0,0,Check-Out,2017-02-24,Jacqueline Reeves,Jacqueline.Reeves@xfinity.com,401-662-7559,************4954
34421,Resort Hotel,0,232,2017,March,11,18,2,5,1,...,Transient,25.5,0,0,Check-Out,2017-03-25,Mary Hardy,Mary_Hardy@xfinity.com,958-252-8125,************9217
52065,City Hotel,1,56,2016,May,23,30,1,0,1,...,Transient-Party,120.0,0,0,Canceled,2016-05-27,Danielle Shepherd,Danielle_Shepherd@mail.com,551-361-5274,************2217
45853,City Hotel,1,98,2015,November,48,23,1,3,1,...,Transient-Party,70.0,0,0,Canceled,2015-11-19,Shirley Hansen,Hansen_Shirley@outlook.com,604-431-2180,************2614
116987,City Hotel,0,167,2017,July,30,24,1,3,2,...,Transient,80.1,0,0,Check-Out,2017-07-28,Christopher Erickson,Christopher_E@zoho.com,224-771-8175,************3203
12167,Resort Hotel,1,33,2017,June,24,16,2,2,2,...,Transient,273.0,0,0,Canceled,2017-05-14,James Mcdonald DDS,James.D@zoho.com,959-527-4245,************5886


In [4]:
df.info

<bound method DataFrame.info of                hotel  is_canceled  lead_time  arrival_date_year  \
0       Resort Hotel            0        342               2015   
1       Resort Hotel            0        737               2015   
2       Resort Hotel            0          7               2015   
3       Resort Hotel            0         13               2015   
4       Resort Hotel            0         14               2015   
...              ...          ...        ...                ...   
119385    City Hotel            0         23               2017   
119386    City Hotel            0        102               2017   
119387    City Hotel            0         34               2017   
119388    City Hotel            0        109               2017   
119389    City Hotel            0        205               2017   

       arrival_date_month  arrival_date_week_number  \
0                    July                        27   
1                    July                        27  

In [5]:
# data cleaninf
# handeling missing value
missing_values_count = df.isnull().sum()
missing_values_count 

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         